In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
# import pandas as pd
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import copy

In [ ]:
# img = mpimg.imread('./dataset/flower/-6.000000_6.000000.png')
# imgplot = plt.imshow(img)
# img.shape

In [ ]:
# import cv2
# img = cv2.imread('./dataset/sphere/-6.000000_6.000000.png',0)
# edges = cv2.Canny(img,60,180)

# print(edges.shape)

# plt.subplot(121),plt.imshow(img,cmap = 'gray')
# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(122),plt.imshow(edges,cmap = 'gray')
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
# plt.imshow(img)

# plt.show()

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
device

In [ ]:
im_s = 128

transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(im_s),
#     transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, ), std=(0.5, )),
])

In [ ]:
def imshow(img):
    # npimg in (channel, height, width)
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray')
    plt.show()

In [ ]:
# Add file name to the label, can be iterated as follow:
# for i, data in enumerate(train_loader):
#     images,labels,paths = data
#     paths = [path.split('/')[-1] for path in paths]
#     print(paths)
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        return super(ImageFolderWithPaths, self).__getitem__(index) + (self.imgs[index][0],)

In [ ]:
train_dataset = ImageFolderWithPaths(
    root='./dataset/',
    transform=transform
)
train_loader = DataLoader(
    train_dataset,
    batch_size=20,
    num_workers=5,
    shuffle=True,
    drop_last=True
)
print(train_dataset.classes)

In [ ]:
# get some random training images
dataiter = iter(train_loader)
quick_images, quick_labels, quick_paths = dataiter.next()

print(quick_images.shape)

# print(quick_images[0].max())
# print(quick_images[0].min())

print(quick_images[:, 0:1, :, :].shape)

# show images
imshow(torchvision.utils.make_grid(quick_images))

# Not sure if normalization is good or not as it darkens the image

In [ ]:
img_shape = (3, im_s, im_s)
cuda = True if torch.cuda.is_available() else False

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.model = nn.Sequential(
            # Pixel Coord + Light Coord + Average Pixel Color
            nn.Linear(2 + 2 + 3, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 3),
            nn.Tanh()
        )
    def forward(self, pixel_coord, light_coord, average_rgb):
        d_in = torch.cat((pixel_coord, light_coord, average_rgb), -1)
        rgb = self.model(d_in)
        return rgb

In [ ]:
# Loss functions
loss = torch.nn.MSELoss()

# Initialize generator and discriminator
DNN = NN()

if cuda:
    DNN.cuda()
    loss.cuda()

In [ ]:
# Optimizers
optimizer = torch.optim.Adam(DNN.parameters(), lr=0.002, betas=(0.9, 0.999))

In [ ]:
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor
CharTensor = torch.cuda.CharTensor if cuda else torch.CharTensor

In [ ]:
# Compute Average Color of All Pixels of All Images
def avg_save(path):
    dic = dict()
    total_image = 0
    for i, data in enumerate(train_loader):
        imgs,labels,paths = data
        # x, y positions of the light
        batch_size = imgs.shape[0]
        num_channel = imgs.shape[1]
        width = imgs.shape[2]
        height = imgs.shape[3]
        total_image += batch_size
        # print(batch_size, num_channel, width, height)
        for k in range(batch_size):
            for i in range(width):
                for j in range(height):
                    key_name = str(i) + '-' + str(j)
                    if key_name in dic:
                        dic[key_name] = imgs[k, :, i, j] + dic[key_name]
                    else:
                        dic[key_name] = imgs[k, :, i, j]
    avg_img = []
    for j in range(height):
        for i in range(width):
            key_name = str(i) + '-' + str(j)
            avg_img += [dic[key_name] / total_image]
    print(total_image)
#     print(avg_img)
    np.save(path, np.array([pix.detach().cpu().numpy() for pix in avg_img]))

In [ ]:
avg_img = avg_save('avg_basketball_128')

In [ ]:
loaded_avg_img = np.load('./avg/avg_basketball_128.npy')
print(loaded_avg_img.shape)
# print(loaded_avg_img)

In [ ]:
plt.figure(figsize=(8,8))
print(loaded_avg_img.shape)
plt.imshow(loaded_avg_img.reshape(img_shape[1],img_shape[2], img_shape[0], order='F') * .5 + .5, cmap='gray')

In [ ]:
def sample_image(n_row, model, loaded_avg_img, lr=0, lc=0):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    width = img_shape[1]
    height = img_shape[2]
    avg_img = torch.from_numpy(loaded_avg_img).to(device)
    avg_img = Variable(avg_img.type(FloatTensor))
    img_tensor = torch.FloatTensor().to(device)
    for k in range(height):
        for j in range(width):
            # Pixel Coord + Light Coord + Average Pixel Color
            avg_value = Variable(avg_img[j + k * width].repeat(1, n_row).view(n_row, 3).type(FloatTensor))
            pixel_coord = Variable(torch.FloatTensor([j/width, k/height])).repeat(1, n_row).view(n_row, 2).to(device)
            # Default sample location is the light in the middle
            light_coord = Variable(torch.FloatTensor([lr, lc])).repeat(1, n_row).view(n_row, 2).to(device)
            generate_pixel = model(pixel_coord, light_coord, avg_value)
#             print(generate_pixel.shape)
            img_tensor = torch.cat((img_tensor, generate_pixel), 0)
    
#     print(img_tensor.shape)
#     imgs = img_tensor.view(img_tensor.size(0), *img_shape)
#     print(img_tensor.cpu().data.detach().numpy().reshape(img_shape[1],img_shape[2],img_shape[0],order='F').shape)
    plt.imshow(img_tensor.cpu().data.detach().numpy().reshape(img_shape[1],img_shape[2],img_shape[0],order='F') * .5 + .5, cmap='gray')
#     print(img_tensor)
#     plt.imshow(loaded_avg_img.reshape(img_shape[1],img_shape[2], img_shape[0], order='F') * .5 + .5, cmap='gray')
    #print(np.round(imgs.data * .5 + .5, 2))
    #plt.imshow(imgs.data)
    #save_image(imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)
    
sample_image(1, DNN, loaded_avg_img)

In [ ]:
# reset params
def reset(m):
    if isinstance(m, nn.Linear):
        m.reset_parameters()
DNN.apply(reset)

In [ ]:
from IPython.display import clear_output

models = []

epochs = 200
for epoch in range(epochs):
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        # we are using paths here instead of labels
        imgs,labels,paths = data
        paths = np.array([path.split('/')[-1][:-4].split('_') for path in paths]).astype(float)
        paths[:, 0] /= 5
        paths[:, 1] -= 5
        paths[:, 1] /= 5
        print(paths)
        positions = torch.from_numpy(paths).to(device)
        avg_img = torch.from_numpy(loaded_avg_img).to(device)
        imgs = imgs.to(device)
        batch_size = imgs.shape[0]
        width = imgs.shape[2]
        height = imgs.shape[3]

        # Configure input
        # Pixel Coord + Light Coord + Average Pixel Color
        avg_img = Variable(avg_img.type(FloatTensor))
        light_coord = Variable(positions.type(FloatTensor))
#         print(avg_img.shape) 
#         print(light_coord.shape)
        total_losses = 0
        for k in range(height):
            for j in range(width):
                # Process the batch at this pixel coord
                
                pixel_value = Variable(imgs[:, :, j, k].view(20, 3).type(FloatTensor))
                
#                 pixel_value = Variable(torch.FloatTensor([1.0, -1.0, -1.0])).repeat(1, 20).view(20, 3).to(device)
                avg_value = Variable(avg_img[j + k * width].repeat(1, 20).view(20, 3).type(FloatTensor))
                #avg_value = Variable(torch.FloatTensor([0.0])).repeat(1, 20).view(20, 1).to(device)
                
                pixel_coord = Variable(torch.FloatTensor([j/width, k/height])).repeat(1, 20).view(20, 2).to(device)
                
                generate_pixel = DNN(pixel_coord, light_coord, avg_value)
                                
                # Image with all lighting conditions
                total_loss = loss(generate_pixel, pixel_value)
                total_losses += total_loss.item()
#                 if j == width/2 or k == width/2:
#                     print(
#                         "[Epoch %d/%d] [Batch %d/%d] [Pixel %d-%d] [Loss: %f]"
#                         % (epoch + 1, epochs, i + 1, len(train_loader), j, k, total_loss)
#                     )
                total_loss.backward()
        optimizer.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [Total loss: %f]"
            % (epoch + 1, epochs, i + 1, len(train_loader), total_losses)
        )
        
        clear_output(wait=True)
        sample_image(1, DNN, loaded_avg_img)
        plt.show()
    if (epoch + 1) % 50 == 0:
        models += [copy.deepcopy(DNN)]

#         batches_done = epoch * len(dataloader) + i
#         if batches_done % opt.sample_interval == 0:
#             sample_image(n_row=10, batches_done=batches_done)

In [ ]:
from ipywidgets import interact

def test(lr, lc, index):
#     sample_image(1, models[index], lr, lc)
    sample_image(1, DNN, loaded_avg_img, lr, lc)
    
# interact(test, lr=(-1, 1, .1), lc=(-1, 1, .1), index=(0, 3, 1))
interact(test, lr=(-1, 1, .1), lc=(-1, 1, .1), index=(0, 3, 1))

In [ ]:
def load_model(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False

    model.eval().to(device)
    return model

In [ ]:
def save_model(model, path):
    checkpoint = {'model': model,
          'state_dict': DNN.state_dict(),
          'optimizer' : optimizer.state_dict()}
    torch.save(checkpoint, path)

In [ ]:
save_model(NN(), './check_pt/trained_basketball.pth')

In [ ]:
model = load_model('./check_pt/trained_basketball_128_200.pth')

In [ ]:
# avg_save('avg_cube_32')
loaded_avg_img = np.load('./avg/avg_basketball_128.npy')

In [ ]:
from ipywidgets import interact

def test(lr, lc):
    sample_image(1, model, loaded_avg_img, lr, lc)
    
interact(test, lr=(-1, 1, .1), lc=(-1, 1, .1))